# 31차시: 멀티 LLM 교차 검증 (Cross-Validation)

## 학습 목표
- 두 개의 LLM (OpenAI, Gemini)에게 **동일한 분석 요청**
- 두 LLM의 **답변 일치/불일치** 분석
- 상위 모델(gpt-5)이 **비교 평가** 수행

## 학습 내용
1. LLM 교차 검증이란?
2. 두 LLM Agent 생성 (동일 Tool)
3. 병렬 분석 실행
4. Judge LLM으로 비교 분석

## 이전 차시 연계
- 28차시: ChatGPT, Gemini API 설정
- 29차시: 재무제표 LLM 분석
- 30차시: LangChain Agent 기초

In [1]:
!pip install -Uq langchain langchain-openai langchain-google-genai langgraph python-dotenv requests beautifulsoup4 lxml finance-datareader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 12.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires go

In [2]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dotenv import load_dotenv
from IPython.display import display, Markdown

# .env 파일 로드
load_dotenv()

# Colab에서 .env 파일 업로드
try:
    from google.colab import files
    print("[Colab 환경 감지]")
    print("=" * 60)
    print(".env 파일을 업로드해주세요.")
    print()
    uploaded = files.upload()
    load_dotenv('.env')
except ImportError:
    print("[로컬 환경]")
    print("=" * 60)
    print(".env 파일이 현재 디렉토리에 있어야 합니다.")

[Colab 환경 감지]
.env 파일을 업로드해주세요.



Saving .env to .env


In [3]:
# API 키 확인
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY')

print(f"[OpenAI API 키 로드 완료]: {OPENAI_API_KEY[:10]}...")
print(f"[Gemini API 키 로드 완료]: {GEMINI_API_KEY[:10]}...")

[OpenAI API 키 로드 완료]: sk-FeQOBhF...
[Gemini API 키 로드 완료]: AIzaSyBARt...


---
## 1. LLM 교차 검증이란?

### 개념
여러 LLM에게 **동일한 질문**을 하고, 답변을 **비교 분석**하여 신뢰도를 높이는 방법

### 왜 필요한가?
- LLM마다 학습 데이터, 추론 방식이 다름
- 단일 LLM의 답변만으로는 **편향** 가능성 존재
- 여러 애널리스트의 의견을 비교하는 것과 동일한 원리

### 구조
```
사용자: "삼성전자를 종합 분석해줘"
       ↓
┌─────────────────┐    ┌─────────────────┐
│   OpenAI Agent                   │    │   Gemini Agent                   │
│   (gpt-5-mini)                       │    │(gemini-2.5-flash)                │
└────────┬────────┘    └────────┬────────┘
                       ↓                                                   ↓
                [분석 결과 A]                               [분석 결과 B]
                       └──────────┬───────────┘
                                                   ↓
                                   ┌───────────────┐
                                   │        Judge LLM             │
                                   │            (gpt-5)                │
                                   └───────┬───────┘
                                                        ↓
                                     [일치/불일치 분석 리포트]
```


---
## 2. Tool 정의 (공통)

두 Agent가 동일한 Tool을 사용합니다.

In [5]:
# 공통 설정
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

def get_company_name(stock_code):
    """종목코드로 회사명 조회"""
    url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"
    try:
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.text, 'html.parser')
        name_tag = soup.select_one('div.wrap_company h2 a')
        if name_tag:
            return name_tag.text.strip()
        return stock_code
    except Exception:
        return stock_code

In [6]:
from langchain.tools import tool

# Tool 1: 재무제표
@tool
def fetch_financial_data(stock_code: str) -> str:
    """
    종목코드로 네이버 금융에서 재무제표를 가져옵니다.
    예시: fetch_financial_data("005930") -> 삼성전자 재무제표
    """
    print(f"  [Tool] 재무제표 수집: {stock_code}")
    url = f"https://finance.naver.com/item/main.nhn?code={stock_code}"

    try:
        tables = pd.read_html(url, encoding='euc-kr')
        financial_table = None
        for table in tables:
            table_str = str(table.columns) + str(table.values)
            if '매출액' in table_str or '영업이익' in table_str:
                financial_table = table
                break

        if financial_table is None:
            return f"{stock_code}: 재무제표를 찾을 수 없습니다."

        company_name = get_company_name(stock_code)
        lines = [f"[{company_name} 재무제표]", financial_table.to_string()]
        return "\n".join(lines)
    except Exception as e:
        return f"재무제표 수집 오류: {e}"

In [7]:
# Tool 2: 뉴스 헤드라인
@tool
def fetch_news_headlines(stock_code: str) -> str:
    """
    종목코드로 해당 기업의 최신 뉴스 헤드라인을 가져옵니다.
    """
    print(f"  [Tool] 뉴스 수집: {stock_code}")
    url = f"https://finance.naver.com/item/news.naver?code={stock_code}"

    try:
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.text, 'html.parser')

        news_items = soup.select('table.type5 tr')
        headlines = []

        for item in news_items[:7]:
            title_tag = item.select_one('a.tit')
            date_tag = item.select_one('td.date')
            if title_tag:
                title = title_tag.get_text(strip=True)
                date = date_tag.get_text(strip=True) if date_tag else ""
                headlines.append(f"- [{date}] {title}")

        if not headlines:
            return f"{stock_code}: 뉴스를 찾을 수 없습니다."

        company_name = get_company_name(stock_code)
        return f"[{company_name} 최신 뉴스]\n" + "\n".join(headlines)
    except Exception as e:
        return f"뉴스 수집 오류: {e}"

In [8]:
# Tool 3: 주가 데이터
@tool
def fetch_stock_price(stock_code: str) -> str:
    """
    종목코드로 최근 30일간 주가 데이터를 가져옵니다.
    """
    print(f"  [Tool] 주가 수집: {stock_code}")

    try:
        import FinanceDataReader as fdr

        end_date = datetime.now()
        start_date = datetime.now() - timedelta(days=45)

        df = fdr.DataReader(stock_code, start_date, end_date)

        if df.empty:
            return f"{stock_code}: 주가 데이터를 찾을 수 없습니다."

        df = df.tail(30)

        company_name = get_company_name(stock_code)
        latest_price = df['Close'].iloc[-1]
        price_30d_ago = df['Close'].iloc[0]
        return_30d = (latest_price - price_30d_ago) / price_30d_ago * 100
        high_30d = df['High'].max()
        low_30d = df['Low'].min()

        lines = [
            f"[{company_name} 주가 현황 (최근 30거래일)]",
            f"- 현재가: {latest_price:,.0f}원",
            f"- 30일 수익률: {return_30d:+.2f}%",
            f"- 30일 최고가: {high_30d:,.0f}원",
            f"- 30일 최저가: {low_30d:,.0f}원",
        ]

        return "\n".join(lines)
    except Exception as e:
        return f"주가 수집 오류: {e}"

In [9]:
# 공통 Tool 목록
tools = [fetch_financial_data, fetch_news_headlines, fetch_stock_price]
print(f"[공통 Tool]: {[t.name for t in tools]}")

[공통 Tool]: ['fetch_financial_data', 'fetch_news_headlines', 'fetch_stock_price']


---
## 3. 두 LLM Agent 생성

In [10]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

print("[LLM 모델 초기화]")
print("=" * 60)

# 분석용 모델 (하위 모델)
model_openai = init_chat_model("gpt-5-mini", model_provider="openai")
print("Agent 1: gpt-5-mini (OpenAI)")

model_gemini = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
print("Agent 2: gemini-2.5-flash (Google)")

# 비교 분석용 모델 (상위 모델)
model_judge = init_chat_model("gpt-5", model_provider="openai")
print("Judge: gpt-5 (OpenAI)")

[LLM 모델 초기화]
Agent 1: gpt-5-mini (OpenAI)
Agent 2: gemini-2.5-flash (Google)
Judge: gpt-5 (OpenAI)


In [11]:
# 시스템 프롬프트 (두 Agent 공통)
system_prompt = """당신은 15년 경력의 CFA 자격증을 보유한 투자 분석 전문가입니다.

사용자 질문에 따라 적절한 도구를 선택하여 분석합니다:
- 재무제표 분석 요청 → fetch_financial_data
- 뉴스/이슈 확인 요청 → fetch_news_headlines
- 주가 추이 확인 요청 → fetch_stock_price
- "종합 분석" 요청 → 3개 도구 모두 활용

분석 결과를 종합하여 다음을 포함한 인사이트를 제공합니다:
- 재무 상태 평가
- 최근 이슈 및 시장 반응
- 주가 추이와 투자 포인트
- 리스크 요인

한국어로 답변합니다."""

In [12]:
# 두 Agent 생성 (동일 Tool, 다른 LLM)
agent_openai = create_agent(
    model=model_openai,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)
print("[OpenAI Agent 생성 완료]")

agent_gemini = create_agent(
    model=model_gemini,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=InMemorySaver()
)
print("[Gemini Agent 생성 완료]")

[OpenAI Agent 생성 완료]
[Gemini Agent 생성 완료]


---
## 4. 병렬 분석 실행

동일한 질문을 두 Agent에게 전달하고 답변을 수집합니다.

In [13]:
def analyze_with_both_agents(question):
    """두 Agent에게 동일한 질문을 하고 답변 수집"""
    print(f"\n[질문] {question}")
    print("=" * 60)

    config_openai = {"configurable": {"thread_id": "openai_session"}}
    config_gemini = {"configurable": {"thread_id": "gemini_session"}}

    # OpenAI Agent 분석
    print("\n[1] OpenAI (gpt-5-mini) 분석 중...")
    result_openai = agent_openai.invoke(
        {"messages": [{"role": "user", "content": question}]},
        config_openai
    )
    answer_openai = result_openai["messages"][-1].content

    # Gemini Agent 분석
    print("\n[2] Gemini (gemini-2.5-flash) 분석 중...")
    result_gemini = agent_gemini.invoke(
        {"messages": [{"role": "user", "content": question}]},
        config_gemini
    )
    answer_gemini = result_gemini["messages"][-1].content

    return answer_openai, answer_gemini

In [14]:
# 삼성전자 종합 분석 요청
question = "삼성전자(005930)를 종합 분석해줘. 재무제표, 최근 뉴스, 주가 추이를 모두 확인해줘."

answer_openai, answer_gemini = analyze_with_both_agents(question)


[질문] 삼성전자(005930)를 종합 분석해줘. 재무제표, 최근 뉴스, 주가 추이를 모두 확인해줘.

[1] OpenAI (gpt-5-mini) 분석 중...
  [Tool] 재무제표 수집: 005930
  [Tool] 뉴스 수집: 005930
  [Tool] 주가 수집: 005930

[2] Gemini (gemini-2.5-flash) 분석 중...
  [Tool] 재무제표 수집: 005930
  [Tool] 뉴스 수집: 005930
  [Tool] 주가 수집: 005930


In [15]:
# OpenAI 분석 결과
print("[OpenAI (gpt-5-mini) 분석 결과]")
print("=" * 60)
display(Markdown(answer_openai))

[OpenAI (gpt-5-mini) 분석 결과]


요청하신 대로 삼성전자(005930)를 재무제표·뉴스·주가(최근 30일) 데이터를 종합하여 분석했습니다. 주요 데이터는 도구에서 제공된 값을 기준으로 정리합니다.

요약(핵심 숫자)
- 현재가(도구 기준): 139,000원
- 최근 30일 등락: +35.21% (30일 고가 144,500원 / 저가 99,300원)
- 2024년(연결, IFRS) 주요 실적(도구 제공)
  - 매출액: 3,008,709 (단위: 백만원)
  - 영업이익: 327,260
  - 당기순이익: 344,514
  - EPS(원): 4,950
  - PER(2024 기준): 10.75
  - BPS(원): 57,981
  - PBR(2024 기준): 0.92
  - 주당배당금: 1,446원 / 배당수익률(2024): 약 2.72% / 배당성향: 29.18%
- 2023년 대비 회복: 2023년 영업이익·순이익이 크게 떨어졌다가 2024년에 실적이 회복됨(영업이익·순이익 증가, ROE도 개선).

1) 재무 상태 평가
- 수익성 회복: 2024년 매출·영업이익·순이익 모두 2023년에 비해 크게 회복되었음. 영업이익과 순이익의 회복은 반도체(특히 메모리) 업황 개선과 제품 믹스 영향 가능성.
- 건전성: BPS 기준으로 순자산이 늘어나고 있으며 PBR 0.92(2024)는 장부가 대비 주가가 저평가된 것으로 해석될 수 있음(단, 2025 추정치는 PBR이 상승 전망). 배당정책은 안정적—배당성향 약 29%로 현금환원에 우호적.
- 수익성 지표: ROE가 2023년 낮은 수준에서 2024년에 되돌림(약 9%)—완전한 고수익주 수준은 아니지만 대형 제조업 대비 안정적 회복세.
- 현금흐름/배당: 주당배당금 지속·소폭 증가. 현금창출력이 정상화되면 추가적인 자사주 소각·특별배당 가능성도 관찰 포인트.

2) 최근 이슈 및 시장 반응
- 도구에서 제공된 뉴스 헤드라인 검색 결과로는 최신 기사(헤드라인)를 찾을 수 없었습니다(검색 결과 없음). 즉각 확인 가능한 특정 뉴스 이벤트는 도구에서 포착되지 않았습니다.
- 그럼에도 불구하고 주가는 최근 30일간 +35% 급등—뉴스가 포착되지 않았더라도 시장의 수급·섹터 모멘텀(예: 메모리 업황 개선 기대, AI 수요 관련 수혜 기대, 외국인/기관 매수, ETF 흐름 등)이나 기술적 반등에 의해 상승했을 가능성이 큽니다.
- 투자자 관점에서 확인해야 할 이벤트: 분기 실적 발표, 경영진 가이던스(특히 메모리·파운드리·시스템 반도체 관련), 주요 고객 수주·협력 소식, 반도체 가격(특히 DRAM/NAND) 추이, 글로벌 경기/수요지표, 지정학 리스크(미·중), 대규모 자본정책(배당/자사주/투자) 공시.

3) 주가 추이와 투자 포인트
- 최근 모멘텀: 30일 고/저(144,500 / 99,300)와 +35% 상승은 단기적 강세. 단기 투자자는 모멘텀을 활용할 수 있으나, 변동성 커질 수 있음.
- 밸류에이션: 도구의 2024년 기준 PER ~10.8, PBR ~0.92는 역사적·동종업종 대비 저평가로 볼 여지. 다만 2025 추정치(도구)상 PER·PBR이 상승(시장이 가격을 미리 반영)되어 향후 주가 기대가 반영된 면이 있음.
- 배당투자 관점: 배당수익률 약 2.7%로 안정적 현금흐름을 중시하는 배당 포트폴리오의 보완 자산으로 적합.
- 투자 아이디어
  - 매수 근거: 실적 회복(2024) 확인, 저평가(PBR<1)·안정적 배당, 반도체 업황(특히 AI 관련 수요)으로 추가 상향 여지.
  - 보수적 접근: 최근 급등에 따른 단기 차익실현·조정 가능성 감안, 분할 매수(달러코스트·시간분할) 권장.

4) 주요 리스크 요인
- 업황 사이클 리스크: 삼성전자의 핵심 수익원은 메모리(및 반도체). 메모리 가격·수요는 매우 사이클릭하여 실적·이익 변동성이 큼.
- 경쟁 심화·기술 리스크: 글로벌 파운드리·시스템 경쟁, 중국 제조업 강화, 기술격차 유지 필요성.
- 거시·정치 리스크: 글로벌 경기 둔화, 환율 변동, 미·중 기술·무역 규제(수출 규제 등)로 인한 영업 영향.
- 자본투입(CAPEX) 부담: 파운드리·최첨단 투자에 대규모 CAPEX 필요—단기 현금흐름·순이익에 부담될 수 있음.
- 주가 과열 리스크: 최근 30일 급등에 따른 단기 조정 가능성(특히 펀더멘털 개선 속도보다 기대가 빠르게 반영된 경우).

실무적 제언(투자 체크리스트)
- 보유·매수 여부 판단 시 확인할 항목
  1) 다음 분기(또는 분기 컨퍼런스콜) 영업이익·메모리 ASP(평균판매단가) 가이던스
  2) 파운드리 및 시스템 부문 수주/수요 흐름
  3) 기관·외국인 수급 흐름 및 대형 눌림매수·자사주 공시
  4) 기술·정책 이슈(미·중 규제, 주요 고객 현황)
  5) 목표 비중·손절 기준(예: 목표주가 대비 몇 % 이상 상승하면 일부 이익 실현 등)

결론
- 재무제표(도구 기준)는 2024년에 실적이 의미있게 회복되었음을 보여주며, 배당·자산가치 측면에서 투자 매력도가 존재합니다. 다만 삼성전자의 실적은 메모리 등 업황에 크게 좌우되므로 향후 업황(메모리 가격, AI 수요 등)과 수급(기관/외국인) 변화를 면밀히 모니터링해야 합니다.
- 현재 주가는 단기 강한 상승을 보였으므로, 단기 매매자는 변동성과 조정 리스크를 고려한 포지셔닝(분할 매수/부분 이익실현)을 권장하고, 장기 가치투자자는 밸류에이션(현재 PBR·PER)·배당과 업황 개선 시나리오를 바탕으로 분산·비중 관리하면서 접근하는 것이 합리적입니다.

원하시면 다음 작업을 추가로 해드리겠습니다.
- 최근 3개월·1년 기술적 차트(지지/저항, 이동평균) 분석
- 메모리(DDR/NAND) 가격·재고 지표와 삼성전자 실적 연동 분석
- 기관·외국인·개인 수급(일별·주별) 데이터 확인

어떤 추가 분석을 원하시나요?

In [16]:
# Gemini 분석 결과
print("[Gemini (gemini-2.5-flash) 분석 결과]")
print("=" * 60)
display(Markdown(answer_gemini[0]['text']))

[Gemini (gemini-2.5-flash) 분석 결과]


CFA 자격증을 보유한 투자 분석 전문가로서 삼성전자(005930)에 대한 종합 분석 결과를 말씀드립니다.

### 재무 상태 평가

삼성전자는 2023년에 매출액, 영업이익, 당기순이익이 전년 대비 크게 감소하며 어려운 한 해를 보냈습니다. 이는 글로벌 경기 둔화와 반도체 업황 부진의 영향이 컸던 것으로 보입니다.

*   **매출액**: 2022년 302조 2,314억 원에서 2023년 258조 9,355억 원으로 감소했으나, 2024년에는 300조 8,709억 원, 2025년에는 329조 1,027억 원으로 상당한 회복 및 성장이 예상됩니다.
*   **영업이익**: 2022년 43조 3,766억 원에서 2023년 6조 5,670억 원으로 급감했으나, 2024년에는 32조 7,260억 원, 2025년에는 40조 1,605억 원으로 큰 폭의 개선이 전망됩니다.
*   **당기순이익**: 2022년 55조 6,541억 원에서 2023년 15조 4,871억 원으로 감소했으나, 2024년에는 34조 4,514억 원, 2025년에는 39조 8,255억 원으로 회복세가 예상됩니다.
*   **수익성 지표 (영업이익률, 순이익률, ROE)**: 2023년에는 크게 하락했으나, 2024년과 2025년에는 점진적으로 개선될 것으로 보입니다.
*   **안정성 지표 (부채비율, 당좌비율, 유보율)**: 부채비율은 25~27% 수준으로 안정적이며, 당좌비율은 180~210% 수준으로 양호한 유동성을 유지하고 있습니다. 유보율 또한 매우 높고 꾸준히 증가하고 있어, 전반적인 재무 건전성은 매우 우수한 것으로 평가됩니다.
*   **EPS**: 2023년 2,131원에서 2024년 4,950원, 2025년 5,727원으로 크게 증가할 것으로 예상되어, 주당 순이익 측면에서도 회복 기대감이 높습니다.

종합적으로 볼 때, 삼성전자는 2023년 실적 부진을 겪었지만, 2024년부터는 강력한 실적 회복이 예상되며 재무 건전성은 여전히 매우 탄탄합니다.

### 최근 이슈 및 시장 반응

최신 뉴스 헤드라인을 확인한 결과, 현재 시점에서 삼성전자에 대한 특정 주요 뉴스는 조회되지 않았습니다. 이는 최근 시장에 큰 영향을 미칠 만한 개별적인 공시나 보도가 없었음을 의미할 수 있습니다. 그러나 주가 추이를 보면 시장은 긍정적인 전망을 선반영하고 있는 것으로 보입니다.

### 주가 추이와 투자 포인트

삼성전자의 최근 30거래일 주가 흐름은 매우 인상적입니다.

*   **현재가**: 139,000원
*   **30일 수익률**: +35.21%
*   **30일 최고가**: 144,500원
*   **30일 최저가**: 99,300원

최근 한 달간 주가가 최저점(99,300원) 대비 크게 상승하여 현재 139,000원에 거래되고 있으며, 30일 최고가에 근접한 수준입니다. 이는 2024년과 2025년의 실적 회복에 대한 시장의 강한 기대감이 반영된 결과로 해석됩니다. 특히 반도체 업황의 바닥 통과 및 인공지능(AI) 관련 수요 증가에 대한 기대감이 주가 상승을 견인하고 있는 것으로 보입니다.

**투자 포인트**:
*   **실적 턴어라운드**: 2023년의 부진을 딛고 2024년과 2025년에 걸쳐 큰 폭의 실적 개선이 예상됩니다.
*   **글로벌 경쟁력**: 메모리 반도체, 스마트폰, 파운드리 등 핵심 사업 부문에서 여전히 세계 최고 수준의 경쟁력을 보유하고 있습니다.
*   **AI 및 신기술 성장**: AI 반도체(HBM), 파운드리 기술력 강화 등 미래 성장 동력 확보 노력이 긍정적으로 평가됩니다.

### 리스크 요인

*   **글로벌 경기 변동성**: 여전히 글로벌 경기 둔화 가능성, 고금리 장기화 등의 불확실성이 존재하며, 이는 IT 수요에 직접적인 영향을 미칠 수 있습니다.
*   **반도체 시장의 주기성**: 반도체 산업은 특유의 경기 사이클을 가지고 있어, 예상보다 회복이 지연되거나 다시 하강 국면에 진입할 위험이 있습니다.
*   **환율 변동**: 수출 비중이 높은 기업 특성상 환율 변동에 따른 실적 영향이 큽니다.
*   **경쟁 심화**: 주요 사업 부문에서 기술 경쟁 및 시장 점유율 확보를 위한 경쟁이 더욱 치열해질 수 있습니다.
*   **예상치와의 괴리**: 현재 주가에는 미래 실적 개선 기대감이 상당 부분 반영되어 있으므로, 실제 실적이 시장의 기대치를 하회할 경우 주가 조정이 발생할 수 있습니다.

### 결론

삼성전자는 2023년의 어려운 시기를 지나 2024년부터는 실적 회복과 성장이 기대되는 국면에 있습니다. 견고한 재무 구조와 기술 리더십을 바탕으로 반도체 업황 회복과 AI 시대의 수혜를 입을 잠재력이 높습니다. 최근 주가도 이러한 긍정적 전망을 반영하여 강한 상승세를 보였습니다. 하지만 글로벌 경기 상황, 반도체 시장의 불확실성, 경쟁 심화 등의 리스크 요인도 상존하므로, 투자 결정 시에는 이러한 요인들을 종합적으로 고려하여 신중하게 접근하시는 것이 중요합니다.

---
## 5. Judge LLM으로 비교 분석

상위 모델(gpt-5)이 두 분석 결과의 일치/불일치를 평가합니다.

In [17]:
def compare_analyses(question, answer_a, answer_b):
    """두 분석 결과를 비교 평가"""

    judge_prompt = f"""당신은 금융 분석 전문가입니다.
두 AI 애널리스트(OpenAI, Gemini)의 분석 결과를 비교 평가해주세요.

## 원래 질문
{question}

## OpenAI (gpt-5-mini) 분석
{answer_a}

## Gemini (gemini-2.5-flash) 분석
{answer_b}

---

다음 형식으로 비교 분석해주세요:

## 1. 핵심 결론 비교
| 항목 | OpenAI | Gemini | 일치 여부 |
|------|--------|--------|----------|
(주요 분석 포인트 비교)

## 2. 일치하는 부분
- 두 분석이 동의하는 핵심 포인트

## 3. 불일치하는 부분
- 의견이 다른 부분과 그 이유

## 4. 분석 품질 평가
| 항목 | OpenAI | Gemini |
|------|--------|--------|
| 데이터 활용도 | (평가) | (평가) |
| 논리적 일관성 | (평가) | (평가) |
| 실용적 인사이트 | (평가) | (평가) |

## 5. 종합 판정
- 신뢰도: (높음/중간/낮음)
- 추천 의견: (투자자가 참고할 최종 의견)
"""

    print("\n[Judge LLM (gpt-5) 비교 분석 중...]")
    print("=" * 60)

    response = model_judge.invoke(judge_prompt)
    return response.content

In [18]:
# 비교 분석 실행
comparison_result = compare_analyses(question, answer_openai, answer_gemini)

print("[비교 분석 결과]")
print("=" * 60)
display(Markdown(comparison_result))


[Judge LLM (gpt-5) 비교 분석 중...]
[비교 분석 결과]


## 1. 핵심 결론 비교
| 항목 | OpenAI | Gemini | 일치 여부 |
|------|--------|--------|----------|
| 2023 부진 → 2024 회복 | 2024년 매출·이익 회복 명시 | 2024·2025 회복/성장 전망 수치 제시 | 일치 |
| 최근 30일 주가 | 현재가 139,000원, +35.21%, 고가 144,500/저가 99,300 | 동일 수치 제시 | 일치 |
| 최근 뉴스 반영 | 도구로 헤드라인 없음 | 헤드라인 없음 | 일치(둘 다 미반영) |
| 밸류에이션 해석 | PER ~10.75, PBR ~0.92 주장, 저평가 강조 | 밸류에이션 배수 직접 제시 없음(저평가 판단 명시적 서술 없음) | 불일치(및 OpenAI 수치 오류 가능) |
| 재무 건전성 | 안정적 배당·현금흐름 회복 전망 | 부채비율 25~27%, 당좌비율 180~210% 등 구체 수치(출처 미명) | 부분 일치 |
| 리스크 인식 | 메모리 사이클, CAPEX, 거시·정책 리스크 | 경기·금리, 사이클, 환율, 경쟁 | 일치 |
| 투자전략 제언 | 분할매수·체크리스트 등 실무 팁 풍부 | 거시·업황 모니터링 중심의 포괄적 제언 | 부분 일치 |
| 데이터 출처·투명성 | “도구 제공” 명시, 뉴스 미조회 공개 | 출처 미명, 전문가 자격 표현 | 부분 불일치(투명성은 OpenAI 우위) |

## 2. 일치하는 부분
- 2023년 실적 부진 이후 2024년(및 2025년) 실적 회복 국면이라는 판단
- 최근 30일 주가 급등과 핵심 수치(현재가, 고가/저가, 수익률)
- 모멘텀의 배경으로 메모리 업황 개선과 AI 수요 기대 제시
- 리스크: 반도체 사이클, 거시/환율, 경쟁 심화
- 최근 개별 굵직한 뉴스/공시 부재(두 모델 모두 헤드라인 조회 실패)

## 3. 불일치하는 부분
- OpenAI의 밸류에이션 수치 오류
  - 주어진 수치로 재계산 시: PER = 139,000 ÷ 4,950 ≈ 28.1(제시 10.75와 불일치)
  - PBR = 139,000 ÷ 57,981 ≈ 2.40(제시 0.92와 불일치)
  - DPS 1,446원 기준 배당수익률도 현재가 대비 ≈1.04%(OpenAI 제시 2.72%와 괴리) → 저평가(PBR<1) 주장은 근거 약함
- Gemini의 안정성 지표(부채비율·당좌비율)와 2025 전망 수치
  - 수치 자체는 개연성 있으나 출처/기준 미명확. 예측치 다수 포함(검증 필요)
- 표현·형식
  - OpenAI: 도구 의존과 한계(뉴스 미조회) 투명하게 언급, 실무 체크리스트 강점
  - Gemini: 자격(CFA) 언급 등 신뢰 신호를 주나, 데이터 출처 기재 미흡

## 4. 분석 품질 평가
| 항목 | OpenAI | Gemini |
|------|--------|--------|
| 데이터 활용도 | 중간: 재무·배당·배수·주가를 폭넓게 제시하나 뉴스 미반영 | 중간: 과거·전망 수치 폭넓게 제시하나 출처 부재, 실시간 뉴스 미반영 |
| 논리적 일관성 | 낮음: PER·PBR·배당수익률이 자체 제시 수치와 산술 불일치 | 보통: 서술 흐름 일관. 다만 제시 수치의 근거·정합성 검증 불가 |
| 실용적 인사이트 | 보통+: 체크리스트·전략(분할 매수, 모니터링 포인트) 유용 | 보통: 핵심 포인트·리스크 명료하나 실행 가이드는 비교적 포괄적 |

## 5. 종합 판정
- 신뢰도: 중간
  - 방향성(업황·실적 회복, 주가 모멘텀, 핵심 리스크)은 두 분석이 일치하나, OpenAI의 밸류에이션 산술 오류가 신뢰를 크게 저하시킴. Gemini는 서술은 안정적이지만 출처 미공개와 전망 수치 비중이 높아 검증이 필요.
- 추천 의견:
  - 제시된 숫자는 그대로 의존하지 말고 재검증이 필요함. 특히 PER·PBR·배당수익률은 다음 방식으로 직접 확인 권고:
    - PER = 현재가 ÷ EPS, PBR = 현재가 ÷ BPS, 배당수익률 = DPS ÷ 현재가
  - 공식 출처(삼성전자 IR, DART, KRX/한국거래소, 신뢰 가능한 데이터벤더)로 2024 실적 확정치·배당, 최근 분기 가이던스, DRAM/NAND/HBM ASP 및 재고 추이, 파운드리 수주 동향을 확인할 것.
  - 단기: 최근 30일 급등(≈+35%) 이후 변동성/조정 리스크 유의. 분할 접근과 이벤트 전후 리스크관리(실적발표, 가이던스)가 합리적.
  - 중장기: 메모리/AI 수요 회복, CAPEX·HBM 증설 실행력, 파운드리 경쟁력 개선 여부가 핵심 촉매. 업황 사이클과 환율 민감도 고려해 비중 관리.

---
## 추가 예시: 다른 기업 분석

In [19]:
# SK하이닉스 분석
question2 = "SK하이닉스(000660)의 투자 매력도를 분석해줘."

answer_openai2, answer_gemini2 = analyze_with_both_agents(question2)
comparison_result2 = compare_analyses(question2, answer_openai2, answer_gemini2)

display(Markdown(comparison_result2))


[질문] SK하이닉스(000660)의 투자 매력도를 분석해줘.

[1] OpenAI (gpt-5-mini) 분석 중...
  [Tool] 재무제표 수집: 000660

[2] Gemini (gemini-2.5-flash) 분석 중...
  [Tool] 재무제표 수집: 000660
  [Tool] 뉴스 수집: 000660
  [Tool] 주가 수집: 000660

[Judge LLM (gpt-5) 비교 분석 중...]


## 1. 핵심 결론 비교
| 항목 | OpenAI | Gemini | 일치 여부 |
|------|--------|--------|----------|
| 전반적 투자매력 | 높음(조건부) | 높음(강함) | 일치 |
| 핵심 드라이버 | HBM/AI 서버 수요, 메모리 업황 회복 | HBM 리더십, 메모리 업황 회복, 기술혁신 | 대체로 일치 |
| 밸류에이션 견해 | PER 저평가(6.4배), PBR 1.62배 | 높은 밸류에이션 부담 가능성 | 부분 불일치 |
| 재무·수익성 평가 | 2024년 대폭 개선, ROE 31% | 2024~2025 폭발적 턴어라운드, ROE 고성장 | 대체로 일치(강도는 Gemini가 더 공격적) |
| 리스크 인식 | 사이클/ASP·CAPEX·경쟁·환율·규제 | 경기·경쟁·투자/기술·밸류 부담·환율 | 일치 |
| 제시 데이터 성격 | 과거/최근 연도 수치 중심(출처 미명시) | 과거+2024/2025 전망, 구체 주가·30일 수익률 제시(출처 미명시) | 불일치(데이터 출처/검증성) |
| 투자전략 제안 | 비중 점진 확대, 분할매수·리스크 관리 | 강한 기대 반영 유의, 실적 확인 필요 | 대체로 일치 |

## 2. 일치하는 부분
- 산업/기업 스토리: 메모리 업황 회복과 AI/HBM 수요가 핵심 성장 동력.
- 2023년 부진 → 2024년 이후 실적 회복 기조.
- 주요 리스크: 메모리 사이클 변동성, CAPEX 부담, 경쟁 심화, 환율/거시 변수.
- 투자 접근: 중장기 성장 잠재력은 매력적이나, 변동성·기대 선반영 가능성을 감안한 신중한 접근 필요.

## 3. 불일치하는 부분
- 수치·단위/현실성:
  - OpenAI: 매출 “661,930(단위: 백만원)” 등 단위 해석상 SK하이닉스 규모 대비 과소(단위/스케일 오류 가능). 그럼에도 PER·ROE 등 비율은 내부 일관성은 있으나 근거 데이터의 신뢰가 낮음.
  - Gemini: 2024/2025 실적과 ROE/EPS를 “예상치”로 매우 구체적 제시, 또한 “현재가 744,000원·30일 수익률 41.98%” 등 구체 가격·구간 수익률을 출처 없이 단정. 허위정밀(precision without verification) 위험.
- 밸류에이션 시각:
  - OpenAI는 “저평가”로 단언.
  - Gemini는 “높은 기대가 선반영” 가능성을 강조(밸류에이션 부담).
- 데이터 출처/검증:
  - 양측 모두 출처·기준시점이 명확하지 않음. 특히 Gemini의 주가·단기 수익률은 사실검증 필요성이 큼.

## 4. 분석 품질 평가
| 항목 | OpenAI | Gemini |
|------|--------|--------|
| 데이터 활용도 | 보통: 재무비율·밸류 지표 제시나 단위/규모 불일치 가능. 출처·시점 부재 | 낮음~보통: 과거+전망·주가 지표를 풍부하게 제시했으나 검증/출처 미제시, 허위정밀 위험 |
| 논리적 일관성 | 보통~좋음: 업황→실적→밸류→전략 흐름은 일관 | 보통: 스토리는 일관적이나 사실·전망 구분과 수치 확정적 서술이 혼재 |
| 실용적 인사이트 | 좋음: 리스크/모니터링 포인트·전술(분할매수/리스크 관리) 제안 | 보통: 포인트 명확하나 과도한 수치 확정성과 가격 데이터 신뢰성 이슈로 활용성 저하 |

## 5. 종합 판정
- 신뢰도: 중간
  - 방향성(업황 회복, HBM 수혜, 사이클 리스크)은 두 분석 모두 타당. 다만 제시 수치·주가 데이터의 출처·정합성 문제가 있어 절대치 신뢰도는 제한적.
- 추천 의견:
  - 두 리포트는 “질적 스토리” 참고용으로 활용하되, “정량 수치”는 반드시 재검증 필요. 최근 분기 실적 공시/컨퍼런스콜, 애널리스트 컨센서스(매출·영업이익·HBM 매출 비중·CAPEX/감가), 동종업체(삼성·마이크론) 증설/기술 로드맵, 고객(특히 AI GPU 생태계) 수요 가이던스를 확인할 것.
  - 투자 실행은:
    - 논리: HBM/AI 구조적 성장 vs. 메모리 사이클/경쟁/밸류 부담.
    - 전술: 분할매수·비중관리(예: 포트폴리오 5~10%), 실적/ASP/재고/수율 지표 체크 후 확신 강화 시 비중 조절.
    - 리스크 헷지: 목표가 구간 도달 시 일부 이익실현, 업황 반전 신호(ASP 하락·재고 증가·고객 가이던스 하향) 시 익스포저 축소.

---
## 학습 정리

### 1. LLM 교차 검증 구조
```
질문 → [Agent A (OpenAI)] → 분석 A
     → [Agent B (Gemini)] → 분석 B
               ↓
       [Judge LLM (gpt-5)]
               ↓
        비교 분석 리포트
```

### 2. 모델 구성
| 역할 | 모델 | 용도 |
|------|------|------|
| Agent 1 | gpt-5-mini | 분석 A |
| Agent 2 | gemini-2.5-flash | 분석 B |
| Judge | gpt-5 | 비교 평가 |

### 3. 활용 포인트
- 단일 LLM 편향 방지
- 다양한 관점의 분석 확보
- 신뢰도 높은 투자 의사결정 지원

---

## 모듈 3 완료

| 차시 | 주제 | 핵심 개념 |
|------|------|----------|
| 19-22 | 머신러닝 | Scikit-learn, 회귀/분류, 평가 |
| 23-24 | 딥러닝 | LSTM, 시계열 예측 |
| 25-26 | NLP | 토큰화, 감성분석 |
| 27-28 | LLM API | OpenAI, Gemini 설정 |
| 29 | LLM 활용 | 재무제표 분석 |
| 30 | LangChain | Agent + Memory |
| 31 | 통합 | **멀티 LLM 교차 검증** |

---

### 다음 모듈 예고
- 모듈 4: 분석 자동화 및 대시보드
  - Streamlit 대시보드
  - 자동 리포트 생성
  - 이메일 발송 자동화